# FIT5148 Assignment 2
#### Task C1, Part c

* Due: 24/05/2019
* Tutor: Paras Sitoula, Wednesday 12-2pm
<br>

| Student 1 | Student 2|
|-----------|----------|
| Hitesh Get | Samuel Campbell |

## 1. Import Necessary Modules

In [19]:
import os
import csv
import random
from time import sleep
import json
import geohash as gh
from kafka import KafkaProducer
import random
from datetime import datetime
from pprint import pprint

## 2. Set Working Directory

In [20]:
print("Please enter a valid path containing the climate and hotspot data")
os.chdir(input())

## 3. Read Satellite Data

In [21]:
with open('./hotspot_TERRA_streaming.csv') as file:
    hotspot_terra_stream = list(csv.reader(file))

In [22]:
pprint({
    hotspot_terra_stream[0][0]: hotspot_terra_stream[1][0],
    hotspot_terra_stream[0][1]: hotspot_terra_stream[1][1],
    hotspot_terra_stream[0][2]: hotspot_terra_stream[1][2],
    hotspot_terra_stream[0][3]: hotspot_terra_stream[1][3]
})

{'confidence': '78',
 'latitude': '-37.966',
 'longitude': '145.051',
 'surface_temperature_celcius': '68'}


## 4. Instantiate Producer

In [ ]:
def publish_message(producer_instance, topic_name, key, data):
    """
    This publishing function encodes the data according to the provided `key` (here json-formatted) 
    and character encoding format (here utf-8).
    
    Then, the data are broadcast to the server with the provided `topic_name`.
    """
    try:
        key_bytes = bytes(key, encoding='utf-8')
        producer_instance.send(topic_name, key = key_bytes, value=data)
        producer_instance.flush()
        print('Message published successfully.  ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print("Publish MEssage",str(ex))        

def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer = lambda x: json.dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
    # Set topic   
    topic = 'climate'
    
    print('Publishing records..')
    # Connect the producer
    producer = connect_kafka_producer()
    
    # JSON-format each randomly-selected record and push it to the stream
    for i in random.sample(range(1, len(hotspot_terra_stream)), len(hotspot_terra_stream) - 1):
        hotspot_terra = dict()
        hotspot_terra["hotspot_terra"] = {}
        hotspot_terra["hotspot_terra"]["latitude"] = float(hotspot_terra_stream[i][0])
        hotspot_terra["hotspot_terra"]["longitude"] = float(hotspot_terra_stream[i][1])
        hotspot_terra["hotspot_terra"]["confidence"] = float(hotspot_terra_stream[i][2])
        hotspot_terra["hotspot_terra"]["surface_temperature_celcius"] = float(hotspot_terra_stream[i][3])
        hotspot_terra["Time_Stamp"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        hotspot_terra["sender_id"] = "hotspot_streamer_terra"
        hotspot_terra['geohash'] = gh.encode(hotspot_terra['hotspot_terra']["latitude"], hotspot_terra['hotspot_terra']["longitude"], precision = 5)
        data = hotspot_terra
        #print(data)
        publish_message(producer, topic, 'jsondata', data)
        
        sleep(10)